In [28]:
%matplotlib inline

import xarray as xr
import intake
import util 
import pprint 
import gsw

In [2]:
if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
col

pangeo-cmip6-ESM Collection with 28660 entries:
	> 10 activity_id(s)

	> 23 institution_id(s)

	> 48 source_id(s)

	> 29 experiment_id(s)

	> 86 member_id(s)

	> 23 table_id(s)

	> 190 variable_id(s)

	> 7 grid_label(s)

	> 28660 zstore(s)

	> 59 dcpp_init_year(s)

In [31]:
cat = col.search(experiment_id=['historical'], table_id=['Omon','SImon'], variable_id=['thetao','siconc','so'],member_id='r1i1p1f1',
                grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
165,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN
169,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,Omon,thetao,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN
323,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP/BCC/BCC-CSM2-MR/historical/r1i...,NaN
325,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Omon,thetao,gn,gs://cmip6/CMIP/BCC/BCC-CSM2-MR/historical/r1i...,NaN
586,CMIP,BCC,BCC-ESM1,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP/BCC/BCC-ESM1/historical/r1i1p1...,NaN
588,CMIP,BCC,BCC-ESM1,historical,r1i1p1f1,Omon,thetao,gn,gs://cmip6/CMIP/BCC/BCC-ESM1/historical/r1i1p1...,NaN
838,CMIP,CAMS,CAMS-CSM1-0,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP/CAMS/CAMS-CSM1-0/historical/r1...,NaN
841,CMIP,CAMS,CAMS-CSM1-0,historical,r1i1p1f1,Omon,thetao,gn,gs://cmip6/CMIP/CAMS/CAMS-CSM1-0/historical/r1...,NaN
847,CMIP,CAMS,CAMS-CSM1-0,historical,r1i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP/CAMS/CAMS-CSM1-0/historical/r1...,NaN
903,CMIP,CAS,FGOALS-f3-L,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP/CAS/FGOALS-f3-L/historical/r1i...,NaN


In [8]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
                                cdf_kwargs={'chunks': {}, 'decode_times': False})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 18 group(s)


In [13]:
print(dset_dict['CMIP.AWI.AWI-CM-1-1-MR.historical.Omon.gn'])

<xarray.Dataset>
Dimensions:    (bnds: 2, depth: 46, member_id: 1, ncells: 830305, time: 1980, vertices: 16)
Coordinates:
  * depth      (depth) float64 -0.0 10.0 20.0 30.0 ... 5.4e+03 5.65e+03 5.9e+03
  * time       (time) int64 0 708 1416 2148 ... 1443420 1444152 1444884 1445616
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds, ncells, vertices
Data variables:
    lon_bnds   (ncells, vertices) float64 dask.array<chunksize=(830305, 16), meta=np.ndarray>
    lon        (ncells) float64 dask.array<chunksize=(830305,), meta=np.ndarray>
    lat_bnds   (ncells, vertices) float64 dask.array<chunksize=(830305, 16), meta=np.ndarray>
    time_bnds  (time, bnds) float64 dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat        (ncells) float64 dask.array<chunksize=(830305,), meta=np.ndarray>
    thetao     (member_id, time, depth, ncells) float32 dask.array<chunksize=(1, 2, 46, 830305), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2


In [54]:
for i in dset_dict:
    print(i)
    if 'depth' in dset_dict[i]:
        dset_dict[i] = dset_dict[i].rename({'depth':'lev'})
    
    if dset_dict[i].lev.units == 'centimeters':
        dset_dict[i].lev = dset_dict[i].lev/100.
        dset_dict[i].lev['units'] = 'm'
        
    print(dset_dict[i])

CMIP.AWI.AWI-CM-1-1-MR.historical.Omon.gn
<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 46, member_id: 1, ncells: 830305, time: 1980, vertices: 16)
Coordinates:
  * lev        (lev) float64 -0.0 10.0 20.0 30.0 ... 5.4e+03 5.65e+03 5.9e+03
  * time       (time) int64 0 708 1416 2148 ... 1443420 1444152 1444884 1445616
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds, ncells, vertices
Data variables:
    lon_bnds   (ncells, vertices) float64 dask.array<chunksize=(830305, 16), meta=np.ndarray>
    lon        (ncells) float64 dask.array<chunksize=(830305,), meta=np.ndarray>
    lat_bnds   (ncells, vertices) float64 dask.array<chunksize=(830305, 16), meta=np.ndarray>
    time_bnds  (time, bnds) float64 dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat        (ncells) float64 dask.array<chunksize=(830305,), meta=np.ndarray>
    thetao     (member_id, time, lev, ncells) float32 dask.array<chunksize=(1, 2, 46, 830305), meta=np.ndarray>
Attributes:
    Conv

AttributeError: cannot set attribute 'lev' on a 'Dataset' object. Use __setitem__ styleassignment (e.g., `ds['name'] = ...`) instead of assigning variables.

In [16]:
print(dset_dict)

AttributeError: 'dict' object has no attribute 'variables'

In [35]:
for i in dset_dict:
    print(dset_dict[i].coords)

Coordinates:
  * depth      (depth) float64 -0.0 10.0 20.0 30.0 ... 5.4e+03 5.65e+03 5.9e+03
  * time       (time) int64 0 708 1416 2148 ... 1443420 1444152 1444884 1445616
  * member_id  (member_id) <U8 'r1i1p1f1'
Coordinates:
  * time       (time) float64 15.5 45.0 74.5 ... 6.015e+04 6.018e+04 6.021e+04
  * lev        (lev) float64 5.0 15.0 25.0 ... 4.289e+03 4.807e+03 5.334e+03
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 -81.5 -80.5 -79.5 -78.5 ... 86.5 87.5 88.5 89.5
  * member_id  (member_id) <U8 'r1i1p1f1'
Coordinates:
  * time       (time) float64 15.5 45.0 74.5 ... 6.015e+04 6.018e+04 6.021e+04
  * lev        (lev) float64 5.0 15.0 25.0 ... 4.289e+03 4.807e+03 5.334e+03
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 -81.5 -80.5 -79.5 -78.5 ... 86.5 87.5 88.5 89.5
  * member_id  (member_id) <U8 'r1i1p1f1'
Coordinates:
  * lev        (lev) float64 5.0 15.0 25.0